## Создание продвинутых ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling.

Для начала, установим Yandex Cloud ML SDK.

In [1]:
%pip install yandex-cloud-ml-sdk

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
INFO: pip is looking at multiple versions of yandexcloud to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of yandexcloud to determine which version is compatible with other requirements. This could take a while.
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
   ---------------------------------------- 108.6/108.6 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 4.0/4.0 MB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 2.9/2.9 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 4.3/4.3 MB 6.1 MB/s eta 0:00:00
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
   -----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.0.239 requires langsmith<0.1.0,>=0.0.11, but you have langsmith 0.1.67 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you have protobuf 4.25.6 which is incompatible.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.6 which is incompatible.


In [19]:
import os 
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ['folder_id']
api_key = os.environ['api_key']


sdk = YCloudML(folder_id=folder_id,auth=api_key)
model = sdk.models.completions("yandexgpt",model_version='rc')

In [3]:
thread = sdk.threads.create(name="Test", ttl_days=1, expiration_policy="static")
assistant =  sdk.assistants.create(model, ttl_days=1, expiration_policy="since_last_active")

thread.write("Ты - учитель геометрии в школе. Тебя зовут мисс Радиус. Твоя задача - отвечать на вопросы ученика.")

assistant.update(instruction="Привет! Что такое число пи?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Привет! Число пи (обозначается как $\pi$) — это математическая константа, которая выражает отношение длины окружности к её диаметру. Независимо от размера окружности, это отношение всегда будет одинаковым и приблизительно равно 3,14. Число пи — иррациональное, то есть его десятичное представление бесконечно и не повторяется.


In [4]:
thread.write("А если округлить до целого?")

run = assistant.run(thread)
result = run.wait()

print(result.text)

Если округлить число $\pi$ до целого, получится 3. Однако для точных расчётов такое округление не используется, так как оно значительно снижает точность вычислений. Обычно число $\pi$ округляют до двух знаков после запятой — 3,14. Это более точное значение, которое часто применяется в геометрических задачах.


In [5]:
thread.delete()
assistant.delete()

## Делаем винного ассистента

In [23]:
from glob import glob
from tqdm.auto import tqdm
import pandas as pd

def get_token_count(filename):
    with open(filename,'r',encoding='utf8') as f:
        return len(model.tokenize(f.read()))

d = [
        {
            "File" : fn,
            "Tokens" : get_token_count(fn),
            "Category" : fn.split('/')[1]
        } 
    for fn in glob('source/*/*.md')
    if fn.count('/') > 2
]

df = pd.DataFrame(d)
df

""


In [24]:
glob('source/*/*.md')

['source\\regions\\Абруццо.md',
 'source\\regions\\Азорские острова.md',
 'source\\regions\\Аконкагуа.md',
 'source\\regions\\Алентежу.md',
 'source\\regions\\Апулия.md',
 'source\\regions\\Ар.md',
 'source\\regions\\Атакама.md',
 'source\\regions\\Аустрал.md',
 'source\\regions\\Баден.md',
 'source\\regions\\Базиликата.md',
 'source\\regions\\Байррада.md',
 'source\\regions\\Бержерак.md',
 'source\\regions\\Бордо.md',
 'source\\regions\\Бургенланд.md',
 'source\\regions\\Бургундия.md',
 'source\\regions\\Бюже.md',
 'source\\regions\\Вайрарапа.md',
 'source\\regions\\Валле-д’Аоста.md',
 'source\\regions\\Вашингтон.md',
 'source\\regions\\Венето.md',
 'source\\regions\\Винью Верде.md',
 'source\\regions\\Вирджиния.md',
 'source\\regions\\Вюртемберг.md',
 'source\\regions\\Гайяк.md',
 'source\\regions\\Галисия.md',
 'source\\regions\\Гессенская горная дорога.md',
 'source\\regions\\Гисборн.md',
 'source\\regions\\Дау.md',
 'source\\regions\\Дору.md',
 'source\\regions\\Жюра.md',
 'source